<center>
<img src="../../img/ods_stickers.jpg">
    
## [mlcourse.ai](https://mlcourse.ai) - Open Machine Learning Course

Authors: [Maria Sumarokova](https://www.linkedin.com/in/mariya-sumarokova-230b4054/), and [Yury Kashnitsky](https://www.linkedin.com/in/festline/). Translated and edited by Gleb Filatov, Aleksey Kiselev, [Anastasia Manokhina](https://www.linkedin.com/in/anastasiamanokhina/), [Egor Polusmak](https://www.linkedin.com/in/egor-polusmak/), and [Yuanyuan Pao](https://www.linkedin.com/in/yuanyuanpao/). All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

# <center> Assignment #3 (demo)
## <center>  Decision trees with a toy task and the UCI Adult dataset 

Same assignment as a [Kaggle Kernel](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees) + [solution](https://www.kaggle.com/kashnitsky/a3-demo-decision-trees-solution). Fill in the answers in the [web-form](https://docs.google.com/forms/d/1wfWYYoqXTkZNOPy1wpewACXaj2MZjBdLOL58htGWYBA/edit).

Let's start by loading all necessary libraries:

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.rcParams["figure.figsize"] = (10, 8)
import collections
from io import StringIO

import numpy as np
import pandas as pd
import pydotplus  # pip install pydotplus
import seaborn as sns
from ipywidgets import Image
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz

### Part 1. Toy dataset "Will They? Won't They?"

Your goal is to figure out how decision trees work by walking through a toy problem. While a single decision tree does not yield outstanding results, other performant algorithms like gradient boosting and random forests are based on the same idea. That is why knowing how decision trees work might be useful.

We'll go through a toy example of binary classification - Person A is deciding whether they will go on a second date with Person B. It will depend on their looks, eloquence, alcohol consumption (only for example), and how much money was spent on the first date.

#### Creating the dataset

In [ ]:
# Create dataframe with dummy variables
def create_df(dic, feature_list):
    out = pd.DataFrame(dic)
    out = pd.concat([out, pd.get_dummies(out[feature_list])], axis=1)
    out.drop(feature_list, axis=1, inplace=True)
    return out


# Some feature values are present in train and absent in test and vice-versa.
def intersect_features(train, test):
    common_feat = list(set(train.keys()) & set(test.keys()))
    return train[common_feat], test[common_feat]

In [ ]:
features = ["Looks", "Alcoholic_beverage", "Eloquence", "Money_spent"]

#### Training data

In [ ]:
df_train = {}
df_train["Looks"] = [
    "handsome",
    "handsome",
    "handsome",
    "repulsive",
    "repulsive",
    "repulsive",
    "handsome",
]
df_train["Alcoholic_beverage"] = ["yes", "yes", "no", "no", "yes", "yes", "yes"]
df_train["Eloquence"] = ["high", "low", "average", "average", "low", "high", "average"]
df_train["Money_spent"] = ["lots", "little", "lots", "little", "lots", "lots", "lots"]
df_train["Will_go"] = LabelEncoder().fit_transform(["+", "-", "+", "-", "-", "+", "+"])

df_train = create_df(df_train, features)
df_train

#### Test data

In [ ]:
df_test = {}
df_test["Looks"] = ["handsome", "handsome", "repulsive"]
df_test["Alcoholic_beverage"] = ["no", "yes", "yes"]
df_test["Eloquence"] = ["average", "high", "average"]
df_test["Money_spent"] = ["lots", "little", "lots"]
df_test = create_df(df_test, features)
df_test

In [ ]:
# Some feature values are present in train and absent in test and vice-versa.
y = df_train["Will_go"]
df_train, df_test = intersect_features(train=df_train, test=df_test)
df_train

In [ ]:
df_test

In [ ]:
df_train_y = pd.concat([df_train, y], axis=1)
df_train_y.sort_values('Looks_handsome')

#### Draw a decision tree (by hand or in any graphics editor) for this dataset. Optionally you can also implement tree construction and draw it here.

#### "Hand" calc

$[4,3]$

$S = .985$

In [ ]:
# присоединяем цель для того чтобы было удобно сортировать
df_hand = pd.concat([df_train, y], axis=1)
# удаляем избыточные столбцы-инверсии бинарных фич
df_hand.drop(columns=['Money_spent_little',
                      'Alcoholic_beverage_no', 'Looks_repulsive', 'Eloquence_average'], inplace=True)
# сортируем
df_hand.sort_values('Money_spent_lots')

**Split №1 `Money_spent_lots`** 

$IG = 0.47$

1. left

    $(0, 2)$
    
    $S = 0$

2. right

    $(4, 1)$

    $S = 0.722$

In [ ]:
# Отобразим правую подвыборку

# удаляем строки относящиеся к левой подвыборке
df_hand.drop(index=[1,3], inplace=True)
# удаляем однородную фичу
df_hand.drop(columns=['Money_spent_lots'], inplace=True)
# сортируем
df_hand.sort_values('Looks_handsome')

**Split №2 `Looks_handsome`** 

$IG = 0.32$

1. left

    $(1, 1)$
    
    $S = 1$

2. right

    $(3, 0)$

    $S = 0$

1\. What is the entropy $S_0$ of the initial system? By system states, we mean values of the binary feature "Will_go" - 0 or 1 - two states in total.

In [ ]:
.985

2\. Let's split the data by the feature "Looks_handsome". What is the entropy $S_1$ of the left group - the one with "Looks_handsome". What is the entropy $S_2$ in the opposite group? What is the information gain (IG) if we consider such a split?

In [ ]:
0.128

#### Train a decision tree using sklearn on the training data. You may choose any depth for the tree.

In [ ]:
clf_tree = DecisionTreeClassifier(criterion="entropy", random_state=17)
clf_tree.fit(df_train, y);

#### Additional: display the resulting tree using graphviz. You can use pydot or [web-service](https://www.coolutils.com/ru/online/DOT-to-PNG) dot2png.

In [ ]:
def tree_graph_to_png(tree, feature_names, png_file_to_save):
    tree_str = export_graphviz(
        tree, feature_names=feature_names, filled=True, out_file=None
    )
    graph = pydotplus.graph_from_dot_data(tree_str)
    graph.write_png(png_file_to_save)

In [ ]:
tree_graph_to_png(clf_tree, df_train.columns, 'toy_tree_.png')

![title](toy_tree_.png)

In [ ]:
# Альтернативная реализация

from sklearn.tree import plot_tree

plot_tree(
    clf_tree, feature_names=df_train.columns, filled=True, class_names=["Won't go", "Will go"]
);

### Part 2. Functions for calculating entropy and information gain.

Consider the following warm-up example: we have 9 blue balls and 11 yellow balls. Let ball have label **1** if it is blue, **0** otherwise.

In [ ]:
balls = [1 for i in range(9)] + [0 for i in range(11)]

<img src = '../../img/decision_tree3.png'>

Next split the balls into two groups:

<img src = '../../img/decision_tree4.png'>

In [ ]:
# two groups
balls_left = [1 for i in range(8)] + [0 for i in range(5)]  # 8 blue and 5 yellow
balls_right = [1 for i in range(1)] + [0 for i in range(6)]  # 1 blue and 6 yellow

#### Implement a function to calculate the Shannon Entropy

In [ ]:
import scipy.stats as stats

def entropy(a_list):
    return stats.entropy(pd.Series(a_list).value_counts(normalize = True), base = 2)

Tests

In [ ]:
print(entropy(balls))  # 9 blue и 11 yellow
print(entropy(balls_left))  # 8 blue и 5 yellow
print(entropy(balls_right))  # 1 blue и 6 yellow
print(entropy([1, 2, 3, 4, 5, 6]))  # entropy of a fair 6-sided die

3\. What is the entropy of the state given by the list **balls_left**?

In [ ]:
0.961236604722876

4\. What is the entropy of a fair dice? (where we look at a dice as a system with 6 equally probable states)?

In [ ]:
stats.entropy([1./6]*6, base=2)

In [ ]:
# information gain calculation
def information_gain(root, left, right):
    """ root - initial data, left and right - two partitions of initial data"""
    return entropy(root) - len(left)/len(root) * entropy(left) - len(right)/len(root) * entropy(right)

5\. What is the information gain from splitting the initial dataset into **balls_left** and **balls_right** ?

In [ ]:
information_gain(balls, balls_left, balls_right)

In [ ]:
def information_gains(X, y):
    """Outputs information gain when splitting with each feature"""
    res = []
    for c in X.columns:
        res.append(information_gain(y, y[X[c] == 0], y[X[c] == 1]))
    return res

#### Optional:
- Implement a decision tree building algorithm by calling `information_gains` recursively
- Plot the resulting tree

In [ ]:
EPS = 1e-5


def my_decision_tree(df_train, y, deep_level=0):
    e = entropy(y)
    deep_space = ' ' * 4 * deep_level
    print(deep_space + f"Entropy: {entropy(y)}")
    print(deep_space + f"Samples: {len(y)}")
    if e > EPS:
        igs = information_gains(df_train, y)
        label = df_train.columns[igs.index(max(igs))]
        feature = df_train[label]
        df_left = df_train[feature == 0]
        df_right = df_train[feature == 1]
        y_left = y[feature == 0]
        y_right = y[feature == 1]
        print(deep_space + f"Best feature to split: {feature.name}")
        print(deep_space +
              f"Samples: {len(df_left)} (left) and {len(df_right)} (right)")
        print(deep_space + '_' * 50)
        deep_level += 1
        #if len(df_left):
        print(deep_space + 'LEFT:')
        my_decision_tree(df_left, y_left, deep_level)
        #if len(df_right):
        print(deep_space + 'RIGHT:')
        my_decision_tree(df_right, y_right, deep_level)
        print(deep_space + '='*50)

In [ ]:
my_decision_tree(df_train, y)

### Part 3. The "Adult" dataset

#### Dataset description:

[Dataset](http://archive.ics.uci.edu/ml/machine-learning-databases/adult) UCI Adult (no need to download it, we have a copy in the course repository): classify people using demographical data - whether they earn more than \$50,000 per year or not.

Feature descriptions:

- **Age** – continuous feature
- **Workclass** –  continuous feature
- **fnlwgt** – final weight of object, continuous feature
- **Education** –  categorical feature
- **Education_Num** – number of years of education, continuous feature
- **Martial_Status** –  categorical feature
- **Occupation** –  categorical feature
- **Relationship** – categorical feature
- **Race** – categorical feature
- **Sex** – categorical feature
- **Capital_Gain** – continuous feature
- **Capital_Loss** – continuous feature
- **Hours_per_week** – continuous feature
- **Country** – categorical feature

**Target** – earnings level, categorical (binary) feature.

#### Reading train and test data

In [ ]:
data_train = pd.read_csv("../../data/adult_train.csv", sep=";")

In [ ]:
data_train.Education.unique()

In [ ]:
data_train.tail()

In [ ]:
data_test = pd.read_csv("../../data/adult_test.csv", sep=";")

In [ ]:
data_test.tail()

In [ ]:
# necessary to remove rows with incorrect labels in test dataset
data_test = data_test[
    (data_test["Target"] == " >50K.") | (data_test["Target"] == " <=50K.")
]

# encode target variable as integer
data_train.loc[data_train["Target"] == " <=50K", "Target"] = 0
data_train.loc[data_train["Target"] == " >50K", "Target"] = 1

data_test.loc[data_test["Target"] == " <=50K.", "Target"] = 0
data_test.loc[data_test["Target"] == " >50K.", "Target"] = 1

#### Primary data analysis

In [ ]:
data_test.describe(include="all").T

In [ ]:
data_train["Target"].value_counts()

In [ ]:
fig = plt.figure(figsize=(25, 15))
cols = 5
rows = np.ceil(float(data_train.shape[1]) / cols)
for i, column in enumerate(data_train.columns):
    ax = fig.add_subplot(rows, cols, i + 1)
    ax.set_title(column)
    if data_train.dtypes[column] == np.object:
        data_train[column].value_counts().plot(kind="bar", axes=ax)
    else:
        data_train[column].hist(axes=ax)
        plt.xticks(rotation="vertical")
plt.subplots_adjust(hspace=0.7, wspace=0.2)

#### Checking data types

In [ ]:
data_train.dtypes

In [ ]:
data_test.dtypes

As we see, in the test data, age is treated as type **object**. We need to fix this.

In [ ]:
data_test["Age"] = data_test["Age"].astype(int)

Also we'll cast all **float** features to **int** type to keep types consistent between our train and test data.

In [ ]:
data_test["fnlwgt"] = data_test["fnlwgt"].astype(int)
data_test["Education_Num"] = data_test["Education_Num"].astype(int)
data_test["Capital_Gain"] = data_test["Capital_Gain"].astype(int)
data_test["Capital_Loss"] = data_test["Capital_Loss"].astype(int)
data_test["Hours_per_week"] = data_test["Hours_per_week"].astype(int)

#### Fill in missing data for continuous features with their median values, for categorical features with their mode.

In [ ]:
# choose categorical and continuous features from data

categorical_columns = [
    c for c in data_train.columns if data_train[c].dtype.name == "object"
]
numerical_columns = [
    c for c in data_train.columns if data_train[c].dtype.name != "object"
]

print("categorical_columns:", categorical_columns)
print("numerical_columns:", numerical_columns)

In [ ]:
# we see some missing values
data_train.info()

In [ ]:
# fill missing data

for c in categorical_columns:
    data_train[c].fillna(data_train[c].mode()[0], inplace=True)
    data_test[c].fillna(data_train[c].mode()[0], inplace=True)

for c in numerical_columns:
    data_train[c].fillna(data_train[c].median(), inplace=True)
    data_test[c].fillna(data_train[c].median(), inplace=True)

In [ ]:
# no more missing values
data_train.info()

We'll dummy code some categorical features: **Workclass**, **Education**, **Martial_Status**, **Occupation**, **Relationship**, **Race**, **Sex**, **Country**. It can be done via pandas method **get_dummies**

In [ ]:
data_train = pd.concat(
    [data_train[numerical_columns], pd.get_dummies(data_train[categorical_columns])],
    axis=1,
)

data_test = pd.concat(
    [data_test[numerical_columns], pd.get_dummies(data_test[categorical_columns])],
    axis=1,
)

In [ ]:
set(data_train.columns) - set(data_test.columns)

In [ ]:
data_train.shape, data_test.shape

#### There is no Holland in the test data. Create new zero-valued feature.

In [ ]:
data_test["Country_ Holand-Netherlands"] = 0

In [ ]:
set(data_train.columns) - set(data_test.columns)

In [ ]:
data_train.head(2)

In [ ]:
data_test.head(2)

In [ ]:
X_train = data_train.drop(["Target"], axis=1)
y_train = data_train["Target"]

X_test = data_test.drop(["Target"], axis=1)
y_test = data_test["Target"]

### 3.1 Decision tree without parameter tuning

Train a decision tree **(DecisionTreeClassifier)** with a maximum depth of 3, and evaluate the accuracy metric on the test data. Use parameter **random_state = 17** for results reproducibility.

In [ ]:
tree = DecisionTreeClassifier(max_depth=3, random_state=17)
tree.fit(X_train, y_train)

Make a prediction with the trained model on the test data.

In [ ]:
tree_predictions = tree.predict(X_test)

In [ ]:
accuracy_score(y_test, tree_predictions)

6\. What is the test set accuracy of a decision tree with maximum tree depth of 3 and **random_state = 17**?

In [ ]:
0.8447884036607088

### 3.2 Decision tree with parameter tuning

Train a decision tree **(DecisionTreeClassifier, random_state = 17).** Find the optimal maximum depth using 5-fold cross-validation **(GridSearchCV)**.

In [ ]:
%%time

tree_params = {"max_depth": range(2, 11)}

locally_best_tree = GridSearchCV(DecisionTreeClassifier(random_state=17), tree_params, cv=5, n_jobs=-1, verbose=True)

locally_best_tree.fit(X_train, y_train)

In [ ]:
best_max_depth = locally_best_tree.best_params_['max_depth']
best_max_depth

In [ ]:
locally_best_tree.best_score_

Train a decision tree with maximum depth of 9 (it is the best **max_depth** in my case), and compute the test set accuracy. Use parameter **random_state = 17** for reproducibility.

In [ ]:
tuned_tree = DecisionTreeClassifier(max_depth=best_max_depth, random_state=17)
tuned_tree.fit(X_train, y_train)
tuned_tree_predictions = tuned_tree.predict(X_test)
accuracy_score(y_test, tuned_tree_predictions)

7\. What is the test set accuracy of a decision tree with maximum depth of 9 and **random_state = 17**?

In [ ]:
0.847798046803022

### 3.3 (Optional) Random forest without parameter tuning

Let's take a sneak peek of upcoming lectures and try to use a random forest for our task. For now, you can imagine a random forest as a bunch of decision trees, trained on slightly different subsets of the training data.

Train a random forest **(RandomForestClassifier)**. Set the number of trees to 100 and use **random_state = 17**.

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=17, n_jobs=-1)
rf.fit(X_train, y_train)

Make predictions for the test data and assess accuracy.

In [ ]:
accuracy_score(y_test, rf.predict(X_test))

### 3.4 (Optional) Random forest with parameter tuning

Train a random forest **(RandomForestClassifier)**. Tune the maximum depth and maximum number of features for each tree using **GridSearchCV**. 

In [ ]:
%%time

forest_params = {'max_depth': range(10, 21),
                 'max_features': range(5, 105, 20)}

locally_best_forest = GridSearchCV(RandomForestClassifier(n_estimators=10, random_state=17, n_jobs=-1),
                                   forest_params, cv=3, n_jobs=-1, verbose=True)

locally_best_forest.fit(X_train, y_train)

In [ ]:
f'Best param: {locally_best_forest.best_params_}; Best score: {locally_best_forest.best_score_}'

Make predictions for the test data and assess accuracy.

In [ ]:
accuracy_score(y_test, locally_best_forest.predict(X_test))